<div class='alert alert-info' style='text-align: center'><h1>Standardizing MR Images</h1>
- yet another MR processing notebook -</div>

#### The goal of this notebook is to demonstrate how to normalize a DICOM MR image with minimal loss of information.
#### - We'll load and normalize an MR image and attempt to maintain it's original pixel distribution.

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
from skimage import exposure
import cv2
from scipy import stats
from IPython.display import clear_output

### - Define a LUT function

In [ ]:
# Make a simple linear VOI LUT from the raw (stored) pixel data
def make_lut(pixels, width, center, p_i):
    
    # Slope and Intercept set to 1 and 0 for MR. Get these from DICOM tags instead if using 
    # on a modality that requires them (CT, PT etc)
    slope = 1.0
    intercept = 0.0
    min_pixel = int(np.amin(pixels))
    max_pixel = int(np.amax(pixels))

    # Make an empty array for the LUT the size of the pixel 'width' in the raw pixel data
    lut = [0] * (max_pixel + 1)
    
    # Invert pixels and cent for MONOCHROME1. We invert the specified center so that 
    # increasing the center value makes the images brighter regardless of photometric intrepretation
    invert = False
    if p_i == "MONOCHROME1":
        invert = True
    else:
        center = (max_pixel - min_pixel) - center
        
    # Loop through the pixels and calculate each LUT value
    for pix_value in range(min_pixel, max_pixel):
        lut_value = pix_value * slope + intercept
        voi_value = (((lut_value - center) /  width + 0.5) * 255.0)
        clamped_value = min(max(voi_value, 0), 255)
        if invert:
            lut[pix_value] = round(255 - clamped_value)
        else:
            lut[pix_value] = round(clamped_value)
        
    return lut

In [ ]:
# Apply the LUT to a pixel array
def apply_lut(pixels_in, lut):
    pixels = pixels_in.flatten()
    pixels_out = [0] * len(pixels)
    for i in range(0, len(pixels)):
        pixel = pixels[i]
        if pixel > 0:
            pixels_out[i] = int(lut[pixel])
    return np.reshape(pixels_out, (pixels_in.shape[0],pixels_in.shape[1]))

### - Load an image

In [ ]:
# Load an image
image = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00216/T1wCE/Image-98.dcm')
raw_pixels = image.pixel_array

# Plot the image
plt.figure(figsize= (6,6))
plt.imshow(raw_pixels, cmap='gray');

In [ ]:
# Plot a histogram of the raw pixels
fig, axes = plt.subplots(nrows=1, ncols=1,sharex=False, sharey=False, figsize=(10,4))
plt.title('Pixel Range of raw pixels: ' + str(np.min(raw_pixels)) + '-' + str(np.max(raw_pixels)))
plt.hist(raw_pixels.ravel(), np.max(raw_pixels), (1, np.max(raw_pixels)))
plt.tight_layout()
plt.show()

- The raw pixels have a range of 0 - 737. This image has more than 8 bits of data.
- We can't display a 16 bit image, so we have to bin the pixels .. either with a LUT, or through normalization.

#### - Let's normalize this image five different ways.

#### 1. Create an 'Auto-LUT' image
- Use the raw pixel range to calculate a 'centered' image.
- This is what pydicom and matplotlib do.

In [ ]:
# Calculate the width and center of the pixels to make a LUT
auto_lut_window_width = np.max(raw_pixels)
auto_lut_window_center = (np.max(raw_pixels) - np.min(raw_pixels)) / 2 + np.min(raw_pixels)

lut = make_lut(raw_pixels, auto_lut_window_width, auto_lut_window_center, image.PhotometricInterpretation)
image_autolut = apply_lut(raw_pixels, lut)

#### 2. Use default WindowWidth and WindowCenter DICOM tags
- This is how the modality unit (MR machine) thinks the image should be displayed according to a predefined protocol.
- The problem with these is that they're not consistent and some appear to have been set prior to bone stripping. So they're just wrong in some cases. But they are great in others.

In [ ]:
# Get the DICOM tags for ww/wc and use them to create the LUT
if image.WindowWidth != '' and image.WindowCenter != '':
    window_width = image.WindowWidth
    window_center = image.WindowCenter

lut = make_lut(raw_pixels, window_width, window_center, image.PhotometricInterpretation)
image_default_window = apply_lut(raw_pixels, lut)

### 3. Normalize with code
- This is how we routinely do it, a simple linear transform.

In [ ]:
pixels = raw_pixels - np.min(raw_pixels)
pixels = pixels / np.max(pixels)
image_manual_norm = (pixels * 255).astype(np.uint8)

### 4. Apply CLAHE
- Use CLAHE with default settings on the raw pixels

In [ ]:
image_clahe = exposure.equalize_adapthist(raw_pixels)

### 5. Apply a manual VOI LUT
- Make the image wider and maintain contrast.

In [ ]:
lut_window_width = 350
lut_window_center = 500

lut = make_lut(raw_pixels, lut_window_width, lut_window_center, image.PhotometricInterpretation)
image_lut = apply_lut(raw_pixels, lut)

### - Display the images and histograms after normalization

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2,sharex=False, sharey=False, figsize=(16, 16))
ax = axes.ravel()

ax[0].set_title('Default WW/WC ' + str(image.WindowWidth) + "/" + str(image.WindowCenter))
ax[0].imshow(image_default_window, cmap='gray')

ax[1].set_title('Pixel Range: ' + str(np.min(image_default_window)) + '-' + str(np.max(image_default_window)))
ax[1].hist(image_default_window.ravel(), np.max(image_default_window), (1, np.max(image_default_window)))

ax[2].set_title('Manual Norm ')
ax[2].imshow(image_manual_norm, cmap='gray')

ax[3].set_title('Pixel Range: ' + str(np.min(image_manual_norm)) + '-' + str(np.max(image_manual_norm)))
ax[3].hist(image_manual_norm.ravel(), np.max(image_manual_norm), (1, np.max(image_manual_norm)))

ax[4].set_title('Auto-LUT ' + str(auto_lut_window_width) + '/' + str(auto_lut_window_center))
ax[4].imshow(image_autolut, cmap='gray')

ax[5].set_title('Pixel Range: ' + str(np.min(image_autolut)) + '-' + str(np.max(image_autolut)))
ax[5].hist(image_autolut.ravel(), 254, (1, 255))

ax[6].set_title('CLAHE ')
ax[6].imshow(image_clahe, cmap='gray')

ax[7].set_title('Pixel Range: ' + str(np.min(image_clahe)) + '-' + str(np.max(image_clahe)))
ax[7].hist(image_clahe.ravel(), 254, (0.1,1))


ax[8].set_title('Manual LUT: ' + str(lut_window_width) + '/' + str(lut_window_center))
ax[8].imshow(image_lut, cmap='gray')

ax[9].set_title('Pixel Range: ' + str(np.min(image_lut)) + '-' + str(np.max(image_lut)))
ax[9].hist(image_lut.ravel(), 254, (1, 254))

plt.tight_layout()
plt.show()

- The raw pixels have values within the 0 to 737 range. That's 738 shades of gray
- In the rest of the images, the same distribution shape occurs, but the pixel value ranges are reduced to 0-255 range. 256 shades of gray (this is Lossy)
- Notice the Default WW/WC image is a little brighter .. it's difficult to see on the picture, but the hist is shifted toward the right.
- Manual Normalization and Auto-LUT result in the same image (maybe need some rounding or slight tweaks, but it's almost identical).

#### IMPORTANT !!
- You can see the tumor in the histograms, it's the bump on the left .. a large group of darker pixels that are a different shade then most of the other pix.
- CLAHE removes this by equalizing contrast between nearby pixels, making the soft tumor tissue look more like the normal tissue .. exactly what we don't want.
- By applying a VOI LUT, we've 'widened' the image (more shades of gray), but not lost the contrast. The distribution is basically the same shape, just fatter.
- Notice the bump on the left of the histogram. We didn't change the tumor to match the brain. You can see, the tumor stands out a little more in the LUT image.

It's hard to visualize, let's enlarge the images a little bit.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3,sharex=False, sharey=False, figsize=(16, 10))
ax = axes.ravel()

ax[0].set_title('Norm')
ax[0].imshow(image_manual_norm, cmap='gray')

ax[1].set_title('CLAHE')
ax[1].imshow(image_clahe, cmap='gray')

ax[2].set_title('LUT')
ax[2].imshow(image_lut, cmap='gray')

plt.tight_layout()
plt.show()

### Conclusion:

- Any normalization techniques applied to MR images must maintain the pixel distribution of the raw pixel data.
- Applying equalization or filters that disturb the inherent contrast of a 16 bit image is counter-productive.

- Tumor Object Detection -> https://www.kaggle.com/davidbroberts/brain-tumor-object-detection
- Determining MR image planes -> https://www.kaggle.com/davidbroberts/determining-mr-image-planes
- Determining MR Slice Orientation -> https://www.kaggle.com/davidbroberts/determining-mr-slice-orientation
- Determining DICOM image order -> https://www.kaggle.com/davidbroberts/determining-dicom-image-order
- Reference Lines on MR images -> https://www.kaggle.com/davidbroberts/mr-reference-lines
- Manual VOI LUT on MR images -> https://www.kaggle.com/davidbroberts/manual-voi-lut-on-mr-images
- Export DICOM Images by Plane -> https://www.kaggle.com/davidbroberts/export-dicom-series-by-plane